In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize, StandardScaler
import torch
import torch.nn as nn
import torchvision.datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import csv

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive


drive.mount('/gdrive')
%cd /gdrive/My\ Drive/AdultDataset

Mounted at /gdrive
/gdrive/My Drive/AdultDataset


In [ ]:
# Prepare Data:
def read_demographics_and_labels(data_name, number_of_sensitive_attributes=1):
    data = pd.read_csv(data_name)

    print(data.columns)

    # Shuffle Data
    # data = data.sample(frac=1)

    data['y'] = 1
    if data_name == "adult.data":
      data['y'].values[data['income'].values == '<=50K'] = 0
    else:
      data['y'].values[data['income'].values == '<=50K.'] = 0

    if number_of_sensitive_attributes == 2:
      data['s0'] = 0
      data['s0'].values[(data['gender'].values == 'Male') & (data['race'].values == 'White')] = 1

      data['s1'] = 0
      data['s1'].values[(data['gender'].values != 'Male') & (data['race'].values == 'White')] = 1

      data['s2'] = 0
      data['s2'].values[(data['gender'].values == 'Male') & (data['race'].values != 'White')] = 1

      data['s3'] = 0
      data['s3'].values[(data['gender'].values != 'Male') & (data['race'].values != 'White')] = 1

      S = data[['s0', 's1', 's2', 's3']]

    else:
      data['s0'] = 0
      data['s0'].values[(data['gender'].values == 'Male')] = 1

      data['s1'] = 0
      data['s1'].values[(data['gender'].values != 'Male')] = 1

      S = data[['s0', 's1']]

    S_matrix = S.to_numpy()
    return data[['y']].to_numpy(), S_matrix

def read_data(training_data_name, test_data_name):

    training_data = pd.read_csv(training_data_name)

    test_data = pd.read_csv(test_data_name)

    X_train = training_data.to_numpy()
    X_test = test_data.to_numpy()

    X_train = normalize(X_train, axis=0)
    X_test = normalize(X_test, axis=0)
    # sc = StandardScaler()

    # X_train = np.array(X_train)
    # sc.fit(X_train)
    # X_train = sc.transform(X_train)
    # X_test = sc.transform(X_test)

    intercept = X_train.shape[0] * [1]
    intercept_numpy = np.array(intercept)
    intercept_numpy = intercept_numpy[:, np.newaxis]
    X_train = np.append(X_train, intercept_numpy, axis=1)

    intercept = X_test.shape[0] * [1]
    intercept_numpy = np.array(intercept)
    intercept_numpy = intercept_numpy[:, np.newaxis]
    X_test = np.append(X_test, intercept_numpy, axis=1)

    return X_train, X_test


y_train, S_Train = read_demographics_and_labels('adult.data')

y_test, S_Test = read_demographics_and_labels('adult.test')

print(y_train.shape)
print(S_Train.shape)

print(y_test.shape)
print(S_Test.shape)

# X_Train, X_Test = read_data('AdultTrain2Sensitive.csv', 'AdultTest2Sensitive.csv')

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
(32561, 1)
(32561, 2)
(16281, 1)
(16281, 2)


In [ ]:
group00 = 0
group01 = 0
group10 = 0
group11 = 0

for i in range(y_train.shape[0]):
  if y_train[i][0] == 0 and S_Train[i][0] == 0:
    group00 += 1

  elif y_train[i][0] == 0 and S_Train[i][0] == 1:
    group01 += 1

  elif y_train[i][0] == 1 and S_Train[i][0] == 0:
    group10 += 1

  elif y_train[i][0] == 1 and S_Train[i][0] == 1:
    group11 += 1

print(group00/32561)
print(group01/32561)
print(group10/32561)
print(group11/32561)

print((group00 + group10) / 32561)
print((group10) / (group10 + group11))

0.29458554712693097
0.46460489542704464
0.036208961641227236
0.20460059580479714
0.33079450876815825
0.15036347404667771


In [ ]:
class FERMI(torch.nn.Module):
# class FERMI():

  def __init__(self, X_train, X_test, Y_train, Y_test, S_train, S_test, batch_size=64, epochs=2000, lam=10):

        super(FERMI, self).__init__()

        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test
        self.S_train = S_train
        self.S_test = S_test

        self.batch_size = batch_size
        self.epochs = epochs

        self.n = X_train.shape[0]
        self.d = X_train.shape[1]
        self.m = Y_train.shape[1]
        if self.m == 1:
          self.m = 2

        self.k = S_train.shape[1]

        self.W = nn.Parameter(torch.zeros(self.k, self.m)) # k: Support of sensitive attributes, m: number of labels
        self.theta = nn.Parameter(torch.zeros(self.d, 1))

        sums = self.S_train.sum(axis=0) / self.n
        print(sums)
        self.p_s0 = sums[0]
        self.p_s1 = sums[1]

        print(sums.shape)

        final_entries = []
        for item in sums:
          final_entries.append(1.0 / np.sqrt(item))

        self.P_s = np.diag(sums)

        self.P_s_sqrt_inv = torch.from_numpy(np.diag(final_entries)).double()
        print(self.P_s_sqrt_inv)
        self.lam = lam


  def forward(self, X):
    outputs = torch.mm(X.double(), self.theta.double())
    logits = torch.sigmoid(outputs)
    return logits


  def grad_loss(self, X, Y):
    outputs = torch.mm(X, self.theta.double())
    probs = torch.sigmoid(outputs)
    return torch.matmul(torch.t(X), probs - Y)

  def fairness_regularizer(self, X, S, f_divergence):

    current_batch_size = X.shape[0]
    summation = 0

    Y_hat = torch.sigmoid(torch.matmul(X, self.theta.double()))
    Y_hat0 = 1 - Y_hat

    p_y1 = torch.mean(Y_hat) # P(y = 1): Taking the average of Y_hat
    p_y0 = 1 - p_y1
    torch.mean(Y_hat)
    p_s0 = torch.mean(S[:, 0])
    p_s1 = torch.mean(S)
    # print(Y_hat.shape)
    # print(S[:, 0])
    # print(S[:, 0].shape)
    p_y1s0 = torch.mean(torch.mul(Y_hat, S[:, 0]))
    p_y1s1 = torch.mean(torch.mul(Y_hat, S[:, 1]))
    p_y0s0 = torch.mean(torch.mul(Y_hat0, S[:, 0]))
    p_y0s1 = torch.mean(torch.mul(Y_hat0, S[:, 1]))

    # print(p_y1s0)
    # print(p_y0s1)
    # print(p_y1s1)
    # print(p_y0s0)

    # print(self.W.shape)
    reg = 0
    if f_divergence == 'Chi2':
      term1 = 2 * p_y1s1 * self.W.double()[1][1] - self.p_s1 * p_y1 * self.W.double()[1][1] * self.W.double()[1][1] + self.p_s1 * p_y1 - 2 * p_y1s1
      term2 = 2 * p_y0s0 * self.W.double()[0][0] - self.p_s0 * p_y0 * self.W.double()[0][0] * self.W.double()[0][0] + self.p_s0 * p_y0 - 2 * p_y0s0
      term3 = 2 * p_y1s0 * self.W.double()[1][0] - self.p_s0 * p_y1 * self.W.double()[1][0] * self.W.double()[1][0] + self.p_s0 * p_y1 - 2 * p_y1s0
      term4 = 2 * p_y0s1 * self.W.double()[0][1] - self.p_s1 * p_y0 * self.W.double()[0][1] * self.W.double()[0][1] + self.p_s1 * p_y0 - 2 * p_y0s1
      reg = term1 + term2 + term3 + term4

    elif f_divergence == 'KL':
      term1 = p_y1s1 * self.W.double()[1][1] - self.p_s1 * p_y1 * torch.exp(self.W.double()[1][1] - 1)
      term2 = p_y0s0 * self.W.double()[0][0] - self.p_s0 * p_y0 * torch.exp(self.W.double()[0][0] - 1)
      term3 = p_y1s0 * self.W.double()[1][0] - self.p_s0 * p_y1 * torch.exp(self.W.double()[1][0] - 1)
      term4 = p_y0s1 * self.W.double()[0][1] - self.p_s1 * p_y0 * torch.exp(self.W.double()[0][1] - 1)
      reg = term1 + term2 + term3 + term4
    # print(reg)

    return self.lam * reg

In [ ]:
def fair_training(fermi, batch_size, epochs, initial_epochs = 300, initial_learning_rate = 1, lam=0.1, learning_rate_min = 0.01, learning_rate_max = 0.01, f_divergence='Chi2'):

  X = fermi.X_train
  S_Matrix = fermi.S_train
  Y = fermi.Y_train
  XTest = fermi.X_test
  STest = fermi.S_test
  YTest = fermi.Y_test

  print(X.shape)
  print(S_Matrix.shape)
  print(Y.shape)

  criterion=torch.nn.BCELoss()

  minimizer = torch.optim.SGD([fermi.theta, fermi.W], lr=initial_learning_rate)
  # maximizer = torch.optim.SGD([fermi.W], lr=learning_rate_max)

  # minimizer_track = []
   # maximizer_track = []

  X_total = torch.from_numpy(X).double()
  Y_total = torch.from_numpy(Y).double()

  for ep in range(epochs + initial_epochs):

      if ep % 100 == 99:
        print(ep+1, " epochs:")
        # Test:
        pre_logits = np.dot(XTest, fermi.theta.detach().numpy())
        output_logits = 1/(1 + np.exp(-pre_logits))
        final_preds = output_logits > 0.5
        print(final_preds.shape)

        p = 0.3
        t = p * torch.ones(16281,1)

        random_numbers = torch.bernoulli(t)
        print(random_numbers)
        final_preds = random_numbers * final_preds
        final_preds = final_preds.numpy()

        test = YTest == 1
        acc = final_preds == test
        true_preds = acc.sum(axis=0)
        print("Accuracy: ", true_preds[0] / output_logits.shape[0] * 100, "%")

        final_preds = np.array(final_preds)
        intersections = np.dot(final_preds.T, STest)
        numbers = STest.sum(axis=0)

        group1 = intersections[0][0] / numbers[0]
        group2 = intersections[0][1] / numbers[1]
        print("DP Violation: ", np.abs(group1 - group2))



      number_of_iterations = X.shape[0] // batch_size
      for i in range(number_of_iterations):


          start = i * batch_size
          end = (i+1) * batch_size

          current_batch_X = X[start:end]
          current_batch_Y = Y[start:end]
          current_batch_S = S_Matrix[start:end]

          XTorch = torch.from_numpy(current_batch_X).double()
          logits = fermi(XTorch)
          YTorch = torch.from_numpy(current_batch_Y).double()
          STorch = torch.from_numpy(current_batch_S).double()

          if ep < initial_epochs:
            loss_min = criterion(logits, YTorch)
          else:
            loss_min = criterion(logits, YTorch) + fermi.fairness_regularizer(XTorch, STorch, f_divergence)
          # loss_min = criterion(logits, YTorch)

          minimizer.zero_grad()
          loss_min.backward()

          if ep >= initial_epochs:
            fermi.theta.grad.data.mul_(learning_rate_min / initial_learning_rate) # You can have \eta_w here
            fermi.W.grad.data.mul_(-learning_rate_max / initial_learning_rate) # You can have \eta_w here

          minimizer.step()
  return fermi.theta, fermi.W

In [ ]:
Y_Train, S_Train = read_demographics_and_labels('adult.data', number_of_sensitive_attributes=1)
Y_Test, S_Test = read_demographics_and_labels('adult.test', number_of_sensitive_attributes=1)

print(Y_Train.shape)
print(S_Train.shape)

print(Y_Test.shape)
print(S_Test.shape)

X_Train, X_Test = read_data('AdultTrain2Sensitive.csv', 'AdultTest2Sensitive.csv')

print(X_Train.shape)
print(X_Test.shape)

# fermi_instance = FERMI(X_Train, X_Test, Y_Train, Y_Test, S_Train, S_Test, lam=0.01)

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
(32561, 1)
(32561, 2)
(16281, 1)
(16281, 2)
(32561, 60)
(16281, 60)


In [ ]:
# Run FERMI
fermi_instance = FERMI(X_Train, X_Test, Y_Train, Y_Test, S_Train, S_Test, lam=100)
theta_star, W_star = fair_training(fermi_instance, batch_size = 2, epochs=2000, initial_epochs=2000, initial_learning_rate=1, learning_rate_min=0.01, learning_rate_max=0.01, f_divergence='Chi2')

[0.66920549 0.33079451]
(2,)
tensor([[1.2224, 0.0000],
        [0.0000, 1.7387]], dtype=torch.float64)
(32561, 60)
(32561, 2)
(32561, 1)
100  epochs:
(16281, 1)
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])
Accuracy:  78.55782814323445 %
DP Violation:  0.08179766429100663
200  epochs:
(16281, 1)
tensor([[0.],
        [0.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])
Accuracy:  77.83919906639642 %
DP Violation:  0.0883244105947711
300  epochs:
(16281, 1)
tensor([[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]])
Accuracy:  77.66107732940237 %
DP Violation:  0.09615190975141688
400  epochs:
(16281, 1)
tensor([[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [1.]])
Accuracy:  77.39696578834224 %
DP Violation:  0.09098711680889032
500  epochs:
(16281, 1)
tensor([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
     

torch.Size([16281, 1])
tensor([[0.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]])


In [ ]:
# Test:
pre_logits = np.dot(X_Test, theta_star.detach().numpy())
output_logits = 1/(1 + np.exp(-pre_logits))
final_preds = output_logits > 0.5
test = Y_Test == 1
acc = final_preds == test
true_preds = acc.sum(axis=0)
print(true_preds / output_logits.shape[0])

In [ ]:
number_of_iterations = 5000
step_size = 0.05
theta = np.zeros((X_Train.shape[1], 1))
W = np.zeros(S_Train.shape[1], Y_Train.shape[1])

epochs = 500
batch_size = 128
for _ in range(epochs):
      number_of_iterations = X_Train.shape[0] // batch_size
      for i in range(number_of_iterations):

          start = i * batch_size
          end = (i+1) * batch_size

          current_batch_X = X_Train[start:end]
          current_batch_Y = Y_Train[start:end]
          # current_batch_S = S_Matrix[start:end]

          logits = np.dot(current_batch_X, theta)
          probs = sigmoid(logits)
          g1 = np.dot(current_batch_X.T, (probs - current_batch_Y))

          theta -= step_size * g1

# for i in range(number_of_iterations):
#     if i % 100 == 0:
#         print(i)
#     logits = np.dot(X_Train, theta)
#     probs = sigmoid(logits)
#     g1 = np.dot(X_Train.T, (probs - Y_Train))
#     # g2 = grad2()

#     theta -= step_size * (g1)


TypeError: ignored

In [ ]:
# Test:
pre_logits = np.dot(X_Test, theta)
output_logits = 1/(1 + np.exp(-pre_logits))
final_preds = output_logits > 0.5
test = Y_Test == 1
acc = final_preds == test
true_preds = acc.sum(axis=0)
print(true_preds / output_logits.shape[0])

i = 0
# print(theta_star)

# for item in final_preds:
#  print(item)
print(pre_logits)